### Generation of planned agent trajectories

- This Jupyter notebook implements a system for generating planned trajectory sequences for agent evaluation in the TAU (Task-oriented Agent Understanding) benchmark framework using open source models that fit in a RTX 4090.
- Rather than executing the actual agent steps, this notebook focuses solely on generating a zero shot planned trajectories (e.g. sequence of tools calls to resolve the user query)
- It is tested with `gemma-3-4b-it`, `Meta-Llama-Guard-2-8B` and `Mistral-7B-Instruct-v0.1`
- The notebook will output a couple of `.parquet` files with the tasks instructions, the ground truth function calls and the LLM planned generated function calls

Before launching this notebook, you need load the desired model in the local environment, for example:

1. Start Docker Desktop
2. Write this `docker-composer.yaml` file
```yaml
services:
    vllm-openai:
        deploy:
            resources:
                reservations:
                    devices:
                        - driver: nvidia
                          count: all
                          capabilities:
                              - gpu
        volumes:
            - ~/.cache/huggingface:/root/.cache/huggingface
            - ./models:/models
        environment:
            - HUGGING_FACE_HUB_TOKEN=<YOUR_HF_TOKEN_HERE>
        ports:
            - 8000:8000
        ipc: host
        image: vllm/vllm-openai:latest
        command: --model ${VLLM_MODEL}
```
3. Start the server, for example in Windows:
```powershell
@echo off

set "model1=google/gemma-3-4b-it"
set "model2=mistralai/Mistral-7B-Instruct-v0.1"
set "model3=meta-llama/Meta-Llama-Guard-2-8B"

if "%1"=="" (
    echo Usage: start-vllm.bat ^<model_name^>
    echo Example: start-vllm.bat %model1%
    echo Allowed models:
    echo   - %model1%
    echo   - %model2%
    echo   - %model3%
    goto :eof
)

set "input_model=%1"

if /I "%input_model%"=="%model1%" goto :start_vllm
if /I "%input_model%"=="%model2%" goto :start_vllm
if /I "%input_model%"=="%model3%" goto :start_vllm

echo Error: Invalid model specified: %input_model%
echo Please choose one of the following models:
echo   - %model1%
echo   - %model2%
echo   - %model3%
goto :eof

:start_vllm
set VLLM_MODEL=%input_model%

echo Starting vLLM service with model: %VLLM_MODEL%
echo The OpenAI-compatible API will be available at: http://localhost:8000
echo Press Ctrl+C to stop the service
echo.
docker-compose down
docker-compose up

echo vLLM service has been stopped 
```
4. Then run the script, for example:
```cmd
.\start-vllm.bat google/gemma-3-4b-it
```


NOTE : Prior downloading the model weights from HF, you need to accept the T&Cs

In [1]:
! pip install -r requirements.txt



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: C:\Users\luisv\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
! git clone https://github.com/sierra-research/tau-bench && cd ./tau-bench && pip install -e .


Obtaining file:///C:/Users/luisv/sandbox/agent-eval/src/notebooks/tau-bench
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for tau_bench (pyproject.toml): started
  Building editable for tau_bench (pyproject.toml): finished with status 'done'
  Created wheel for tau_bench: filename=tau_bench-0.1.0-0.editable-py3-none-any.whl size=6046 sha256=0e0ee3c47443f090829f9f7022f692ed1149400e3f9a31ce4e73a0190d98a16b
  Stored in directory: C:\Users\luisv\AppData\Local\Temp\pip-ephem-wheel-cache-hpmps1vw\wheels\d2\25\41\ff0795e4d487d639de

Cloning into 'tau-bench'...

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: C:\Users\luisv\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import litellm
import os
import re
import json
import sys
import logging
import pandas as pd

from tqdm import tqdm
from collections import Counter
from typing import Dict, List, Optional, Any, Tuple, Callable
from concurrent.futures import ThreadPoolExecutor

from tau_bench.envs.retail.tasks import tasks as retail_tasks
from tau_bench.envs.airline.tasks import tasks as airline_tasks

from tau_bench.envs.airline.tools import ALL_TOOLS as airline_tools
from tau_bench.envs.retail.tools import ALL_TOOLS as retail_tools

In [4]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [5]:
RETAIL_DATA_PATH = "retail_eval_results.parquet"
AIRLINE_DATA_PATH = "airline_eval_results.parquet"

In [6]:
AVAILABLE_MODELS = {
    "gemma-3-4b": "openai/google/gemma-3-4b-it",
    "llama-guard": "openai/meta-llama/Meta-Llama-Guard-2-8B", 
    "mistral-7b": "openai/mistralai/Mistral-7B-Instruct-v0.1"
}

# Select model by uncommenting one of these lines
MODEL_NAME = AVAILABLE_MODELS["gemma-3-4b"]
# MODEL_NAME = AVAILABLE_MODELS["llama-guard"]
# MODEL_NAME = AVAILABLE_MODELS["mistral-7b"]

In [7]:
model_configs = {
    "openai/mistralai/Mistral-7B-Instruct-v0.1": {"model": "openai/mistralai/Mistral-7B-Instruct-v0.1", "use_system_prompt": True, "api_base": "http://localhost:8000/v1"},
    "openai/google/gemma-3-4b-it": {"model": "openai/google/gemma-3-4b-it", "use_system_prompt": True, "api_base": "http://localhost:8000/v1"},
    "openai/meta-llama/Meta-Llama-Guard-2-8B": {"model": "openai/meta-llama/Meta-Llama-Guard-2-8B", "use_system_prompt": False, "api_base": "http://localhost:8000/v1"} 
}

In [8]:
model_config = model_configs[MODEL_NAME]

In [9]:
def get_tool_dict(tool_class):
    tool_info = tool_class.get_info()
    function_info = tool_info["function"]
    return {
        "name": function_info["name"],
        "description": function_info["description"],
        "parameters": function_info["parameters"]
    }


def parse_json_plan(response_text: str) -> List[Dict[str, Any]]:
    try:
        json_match = re.search(r'(\[.*\])', response_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(1)
            json_str = json_str.replace('\\"', '"')
            plan = json.loads(json_str)
            if isinstance(plan, list):
                validated_plan = []
                for item in plan:
                    if isinstance(item, dict) and "name" in item and "arguments" in item:
                        validated_plan.append(item)
                    else:
                        logging.warning(f"Skipping invalid item in plan: {item}")
                return validated_plan
            else:
                logging.warning(f"Parsed JSON is not a list: {plan}")
                return []
        else:
            logging.warning("No JSON array found in the response.")
            return []
    except json.JSONDecodeError as e:
        logging.error(f"Error parsing JSON plan: {e}")
        logging.error(f"Raw response: {response_text}")
        return [{"error": f"JSONDecodeError: {e}", "raw_response": response_text}]
    except Exception as e:
        logging.error(f"Unexpected error parsing plan: {e}")
        return [{"error": f"Unexpected error: {e}", "raw_response": response_text}]


def generate_single_shot_plan(
    user_query: str,
    tools: List[Dict[str, Any]],
    model: str,
    api_base: str,
    use_system_prompt: bool 
) -> List[Dict[str, Any]]:
    

    if use_system_prompt:
        system_prompt = f"""You are a planning agent. Your goal is to create a plan to fulfill the user's request using the available functions.
Available functions: {json.dumps(tools, indent=2)}

Respond ONLY with a JSON array outlining the sequence of function calls needed. Each object in the array should have 'function_name' and 'parameters' keys.

Example format:
[
    {{"name": "search_flights", "arguments": {{"origin": "JFK", "destination": "SFO"}}}},
    {{"name": "book_reservation", "arguments": {{"flight_id": "UA123", "user_id": "user123"}}}}
]

If no function calls are needed, return an empty JSON array: [].
Do not include any other text, explanation, or markdown formatting."""

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_query}
        ]
        prompt_type_log = "separate system/user prompts"
    else:
        combined_prompt = f"""Instructions: You are a planning agent. Create a plan to fulfill the user's request using the available functions. Respond ONLY with a JSON array outlining the sequence of function calls needed. Each object in the array should have 'function_name' and 'parameters' keys. If no calls are needed, return an empty JSON array: []. Do not include any other text, explanation, or markdown formatting.

Available functions:
{json.dumps(tools, indent=2)}

Example format:
[
    {{"name": "search_flights", "arguments": {{"origin": "JFK", "destination": "SFO"}}}},
    {{"name": "book_reservation", "arguments": {{"flight_id": "UA123", "user_id": "user123"}}}}
]

---
User request: {user_query}
---

Generate the plan:"""

        messages = [
            {"role": "user", "content": combined_prompt}
        ]
        prompt_type_log = "combined user prompt"

    try:
        print(f"Generating full plan (Model: {model}, Prompt Format: {prompt_type_log})...")
        response = litellm.completion(
            model=model,
            api_base=api_base,
            messages=messages,
            temperature=0.0,
            max_tokens=1024
        )

        if not response or not hasattr(response, 'choices') or not response.choices or response.choices[0].message.content is None:
            print("Warning: Empty response or None content from model.")
            return [{"error": "Empty response from model"}]

        result = response.choices[0].message.content.strip()
        print(f"Model raw response:\n{result}")

        plan = parse_json_plan(result)
        print(f"\nParsed Plan: {plan}")
        return plan

    except Exception as e:
        import traceback
        tb_str = traceback.format_exc()
        print(f"Error during plan generation: {str(e)}\n{tb_str}")
        raw_resp_text = result if 'result' in locals() else "Response not captured"
        return [{"error": str(e), "raw_response": raw_resp_text}]


In [10]:
retail_instructions = [task["instruction"] for task in retail_tasks]
airline_instructions = [task["instruction"] for task in airline_tasks]

retail_gt_calls = [task["actions"] for task in retail_tasks]
airline_gt_calls = [task["actions"] for task in airline_tasks]


airline_tool_dicts = [get_tool_dict(tool_class) for tool_class in airline_tools]
retail_tool_dicts = [get_tool_dict(tool_class) for tool_class in retail_tools]

retail_df = pd.DataFrame({
    'instruction': retail_instructions,
    'ground_truth_calls': retail_gt_calls,
})


retail_df['tools'] = [retail_tool_dicts] * len(retail_df)


airline_df = pd.DataFrame({
    'instruction': airline_instructions,
    'ground_truth_calls': airline_gt_calls,
})

airline_df['tools'] = [airline_tool_dicts] * len(airline_df)

In [11]:
def process_airline_row(args):
    index, instruction = args
    plan = generate_single_shot_plan(
        user_query=instruction,
        tools=airline_tool_dicts,
        model=model_config["model"],
        api_base=model_config["api_base"],
        use_system_prompt=model_config["use_system_prompt"]
    )
    return index, plan

def process_retail_row(args):
    index, instruction = args
    plan = generate_single_shot_plan(
        user_query=instruction,
        tools=retail_tool_dicts,
        model=model_config["model"],
        api_base=model_config["api_base"],
        use_system_prompt=model_config["use_system_prompt"]
    )
    return index, plan


airline_df['airline_planned_calls'] = None
with ThreadPoolExecutor() as executor:
    airline_args = [(i, row['instruction']) for i, row in airline_df.iterrows()]
    for index, plan in tqdm(executor.map(process_airline_row, airline_args), total=len(airline_df),desc="Processing airline instructions"):
        airline_df.at[index, 'airline_planned_calls'] = plan


retail_df['retail_planned_calls'] = None
with ThreadPoolExecutor() as executor:
    retail_args = [(i, row['instruction']) for i, row in retail_df.iterrows()]
    for index, plan in tqdm(executor.map(process_retail_row, retail_args),total=len(retail_df), desc="Processing retail instructions"):
        retail_df.at[index, 'retail_planned_calls'] = plan


13:11:25 - LiteLLM:INFO: utils.py:3075 - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
2025-04-11 13:11:25,436 - INFO - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
13:11:25 - LiteLLM:INFO: utils.py:3075 - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
13:11:25 - LiteLLM:INFO: utils.py:3075 - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
2025-04-11 13:11:25,436 - INFO - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
2025-04-11 13:11:25,437 - INFO - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
13:11:25 - LiteLLM:INFO: utils.py:3075 - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
13:11:25 - LiteLLM:INFO: utils.py:3075 - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
2025-04-11 13:11:25,438 - INFO - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
13:11:25 - LiteLLM:INFO: utils.py:3075 - 


Generating full plan (Model: openai/google/gemma-3-4b-it, Prompt Format: separate system/user prompts)...
Generating full plan (Model: openai/google/gemma-3-4b-it, Prompt Format: separate system/user prompts)...
Generating full plan (Model: openai/google/gemma-3-4b-it, Prompt Format: separate system/user prompts)...
Generating full plan (Model: openai/google/gemma-3-4b-it, Prompt Format: separate system/user prompts)...
Generating full plan (Model: openai/google/gemma-3-4b-it, Prompt Format: separate system/user prompts)...
Generating full plan (Model: openai/google/gemma-3-4b-it, Prompt Format: separate system/user prompts)...
Generating full plan (Model: openai/google/gemma-3-4b-it, Prompt Format: separate system/user prompts)...
Generating full plan (Model: openai/google/gemma-3-4b-it, Prompt Format: separate system/user prompts)...
Generating full plan (Model: openai/google/gemma-3-4b-it, Prompt Format: separate system/user prompts)...
Generating full plan (Model: openai/google/gem

Processing airline instructions:   0%|          | 0/50 [00:00<?, ?it/s]2025-04-11 13:11:27,794 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
13:11:27 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
2025-04-11 13:11:27,799 - INFO - Wrapper: Completed Call, calling success_handler
13:11:27 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
2025-04-11 13:11:27,800 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
13:11:27 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
2025-04-11 13:11:27,802 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
13:11:27 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: google/gemma-3-4b-it
13:11:27 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calcula

Model raw response:
[
  {"name": "get_reservation_details", "arguments": {"reservation_id": "amelia_sanchez_4739_MCO_CLT"}},
  {"name": "cancel_reservation", "arguments": {"reservation_id": "amelia_sanchez_4739_MCO_CLT"}}
]

Parsed Plan: [{'name': 'get_reservation_details', 'arguments': {'reservation_id': 'amelia_sanchez_4739_MCO_CLT'}}, {'name': 'cancel_reservation', 'arguments': {'reservation_id': 'amelia_sanchez_4739_MCO_CLT'}}]
Generating full plan (Model: openai/google/gemma-3-4b-it, Prompt Format: separate system/user prompts)...
Model raw response:
```json
[
  {"name": "search_direct_flight", "arguments": {"origin": "ATL", "destination": "PHL", "date": "2024-05-02"}},
  {"name": "update_reservation_flights", "arguments": {"reservation_id": "ZFA04Y", "cabin": "economy"}}
]
```

Parsed Plan: [{'name': 'search_direct_flight', 'arguments': {'origin': 'ATL', 'destination': 'PHL', 'date': '2024-05-02'}}, {'name': 'update_reservation_flights', 'arguments': {'reservation_id': 'ZFA04Y', 

2025-04-11 13:11:29,184 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
13:11:29 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
2025-04-11 13:11:29,185 - INFO - Wrapper: Completed Call, calling success_handler
13:11:29 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
2025-04-11 13:11:29,187 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
13:11:29 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
13:11:29 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: google/gemma-3-4b-it
2025-04-11 13:11:29,187 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
2025-04-11 13:11:29,188 - INFO - selected model name for cost calculation: google/gemma-3-4b-it
13:11:29 - LiteLLM:INFO: cost_calculator.py:622 - selected mo

...

Model raw response:
```json
[
  {"name": "get_user_id_by_email", "arguments": {"email": "sara_doe_496"}},
  {"name": "get_order_details", "arguments": {"order_id": "#W0000000"}},
  {"name": "modify_pending_order_address", "arguments": {"order_id": "#W0000000", "address1": "123 Main St", "address2": "Apt 1", "city": "New York", "state": "NY", "country": "USA", "zip": "10001"}},
  {"name": "modify_pending_order_items", "arguments": {"order_id": "#W0000000", "item_ids": ["9844888101"], "new_item_ids": ["9844888101"], "payment_method_id": "gift_card_0000000"}},
  {"name": "exchange_delivered_order_items", "arguments": {"order_id": "#W0000000", "item_ids": ["1008292230"], "new_item_ids": ["1008292230"], "payment_method_id": "credit_card_0000000"}},
  {"name": "think", "arguments": {"thought": "Okay, I've modified the address and laptop item. Now I need to exchange the watch for a black dial."}},
  {"name": "cancel_pending_order", "arguments": {"order_id": "#W0000000", "reason": "no longer n

2025-04-11 13:12:16,234 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
13:12:16 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
2025-04-11 13:12:16,237 - INFO - Wrapper: Completed Call, calling success_handler
13:12:16 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
2025-04-11 13:12:16,238 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
13:12:16 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
2025-04-11 13:12:16,238 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
13:12:16 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: google/gemma-3-4b-it
2025-04-11 13:12:16,239 - INFO - selected model name for cost calculation: google/gemma-3-4b-it
13:12:16 - LiteLLM:INFO: cost_calculator.py:622 - selected mo

Model raw response:
```json
[
  {"name": "list_all_product_types"},
  {"name": "calculate", "arguments": {"expression": "200"}},
  {"name": "get_product_details", "arguments": {"product_id": "mechanical_keyboard_0001"}},
  {"name": "get_product_details", "arguments": {"product_id": "mechanical_keyboard_0002"}},
  {"name": "get_product_details", "arguments": {"product_id": "mechanical_keyboard_0003"}},
  {"name": "get_product_details", "arguments": {"product_id": "mechanical_keyboard_0004"}},
  {"name": "get_product_details", "arguments": {"product_id": "mechanical_keyboard_0005"}},
  {"name": "get_product_details", "arguments": {"product_id": "mechanical_keyboard_0006"}},
  {"name": "get_product_details", "arguments": {"product_id": "mechanical_keyboard_0007"}},
  {"name": "get_product_details", "arguments": {"product_id": "mechanical_keyboard_0008"}},
  {"name": "get_product_details", "arguments": {"product_id": "mechanical_keyboard_0009"}},
  {"name": "get_product_details", "argument

2025-04-11 13:12:19,658 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
13:12:19 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
2025-04-11 13:12:19,661 - INFO - Wrapper: Completed Call, calling success_handler
13:12:19 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
2025-04-11 13:12:19,662 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
13:12:19 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
2025-04-11 13:12:19,662 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
13:12:19 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: google/gemma-3-4b-it
2025-04-11 13:12:19,663 - INFO - selected model name for cost calculation: google/gemma-3-4b-it
13:12:19 - LiteLLM:INFO: cost_calculator.py:622 - selected mo

Model raw response:
```json
[
  {"name": "find_user_id_by_email", "arguments": {"email": "user@example.com"}},
  {"name": "get_order_details", "arguments": {"order_id": "#W0000000"}},
  {"name": "modify_pending_order_items", "arguments": {"order_id": "#W0000000", "item_ids": ["1008292230"], "new_item_ids": ["1008292230"], "payment_method_id": "visa_0000000"}},
  {"name": "modify_pending_order_items", "arguments": {"order_id": "#W0000000", "item_ids": ["1008292231"], "new_item_ids": ["1008292231"], "payment_method_id": "visa_0000000"}},
  {"name": "modify_pending_order_items", "arguments": {"order_id": "#W0000000", "item_ids": ["1008292232"], "new_item_ids": ["1008292232"], "payment_method_id": "visa_0000000"}},
  {"name": "modify_pending_order_items", "arguments": {"order_id": "#W0000000", "item_ids": ["1008292233"], "new_item_ids": ["1008292233"], "payment_method_id": "visa_0000000"}},
  {"name": "modify_pending_order_items", "arguments": {"order_id": "#W0000000", "item_ids": ["100829

In [12]:
retail_df = retail_df.drop('tools', axis=1)
airline_df = airline_df.drop('tools', axis=1)

retail_df.to_parquet(RETAIL_DATA_PATH, engine='fastparquet')
airline_df.to_parquet(AIRLINE_DATA_PATH, engine='fastparquet')